In [290]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from prettytable import PrettyTable
from datetime import datetime, timedelta
from tqdm import tqdm
import statsmodels.api as sm
import yfinance as yf
import numpy as np

In [291]:
ANOS_PASSADOS=5

In [292]:
def parse_bcb(codigo_serie, indice):
    # Obtendo a data atual
    hoje = datetime.now()
    
    # Calculando as datas de início e fim
    data_inicio = f"{hoje.day}/{hoje.month}/{hoje.year-ANOS_PASSADOS}"
    data_final = f"{hoje.day}/{hoje.month}/{hoje.year}"

    # Construindo a URL para a API
    URL = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=csv&dataInicial={data_inicio}&dataFinal={data_final}"
    #print(URL)
    # Lendo o CSV do link fornecido
    df = pd.read_csv(URL, sep=";")

    # Convertendo a coluna "data" para o formato datetime
    # Aqui, ajustei o formato da data para corresponder ao formato fornecido: %d/%m/%Y
    df.rename(columns={"data": "date"}, inplace=True)
    df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y')
    df.set_index('date', inplace=True)
    df["valor"] = df["valor"].str.replace(',', '.').astype(float)
    df.rename(columns={"valor": indice}, inplace=True)
    return df


In [293]:
selic_df=parse_bcb(11, "Selic")
ipca_df=parse_bcb(10844, "IPCA")
SELIC=(selic_df.mean().values[0]*22*12)/100
SELIC=round(SELIC,2)
IPCA=(ipca_df.mean().values[0]*12)/100
IPCA=round(IPCA,2)


ANO_DIAS_UTEIS=252

SELIC,IPCA

(0.07, 0.04)

In [294]:
mercado=["^BVSP",]
acoes=["MGLU3","GGBR4"]
acoes=[t+ ".SA" for t in acoes ]
tickers=[t.upper() for t in acoes+mercado]
tickers

['MGLU3.SA', 'GGBR4.SA', '^BVSP']

In [295]:
hoje = datetime.now()
start_date=f"{hoje.year-ANOS_PASSADOS}-{datetime.now().month}-{datetime.now().day}"
data= yf.download(tickers, start=start_date)['Adj Close']

[*********************100%%**********************]  3 of 3 completed


In [296]:
mercado_df=data[tickers[-1]]
acoes_df=data[tickers[:-1]]

retorno_diario = acoes_df.pct_change()
retorno_anual = retorno_diario.mean() * ANO_DIAS_UTEIS
cov_diaria = retorno_diario.cov()
cov_anual = cov_diaria * ANO_DIAS_UTEIS

In [297]:
import statsmodels.api as sm
import numpy as np


def alpha_beta(acoes_df, mercado_df):
    # Calcula os retornos logarítmicos do mercado
    retorno_mkt = np.log(mercado_df/mercado_df.shift(1))
    retorno_mkt = retorno_mkt.dropna()

    alphas = {}
    betas = {}

    for col in acoes_df.columns:
        # Calcula os retornos logarítmicos da ação
        retorno_ativo = np.log(acoes_df[col]/acoes_df[col].shift(1))
        retorno_ativo = retorno_ativo.dropna()

        # Alinha os índices de datas para garantir que estão comparando os mesmos dias
        df = pd.concat([retorno_ativo, retorno_mkt], axis=1).dropna()

        y = df[col]
        X = df[mercado_df.name]
        X = sm.add_constant(X)

        model = sm.OLS(y, X).fit()

        alphas[col] = model.params[0]
        betas[col] = model.params[1]

    return alphas, betas

alphas, betas = alpha_beta(acoes_df, mercado_df)
betas


{'MGLU3.SA': 1.3660708090672546, 'GGBR4.SA': 1.1122895162912738}

In [298]:
def retorno_esperado(beta,taxa_livre_risco, premio_risco):
    re = taxa_livre_risco + beta * (premio_risco-taxa_livre_risco)
    return  re


def vol_anual(data,ticker):
    daily_returns = np.log(data/data.shift(1))
    daily_returns = daily_returns[ticker].dropna()
    return daily_returns.std() * (252**0.5)

vol_anual(acoes_df, "MGLU3.SA")

def sharpe_ratio(data, ticker, retorno_esperado, taxa_livre_risco):
    
    std = vol_anual(data, ticker) 
    # Calcula o Sharpe Ratio
    sharpe = (retorno_esperado - taxa_livre_risco) / std
    
    return sharpe
sharpe_ratio(acoes_df, "MGLU3.SA", re, SELIC)


def vol_anual_ajustada(data,ticker):
    daily_returns = np.log(data/data.shift(1))
    daily_returns = daily_returns[ticker].dropna()
    downside = daily_returns[daily_returns < 0]
    return downside.std() * (252**0.5)

vol_anual_ajustada(acoes_df, "MGLU3.SA")

def sortino_ratio(data, ticker, retorno_esperado, taxa_livre_risco):
    std = vol_anual_ajustada(data,ticker)
    
    # Calcula o Sharpe Ratio
    sortino = (retorno_esperado - taxa_livre_risco) / std
    
    return sortino
sortino_ratio(acoes_df, "MGLU3.SA", re, SELIC)

retorno_carteira = []
peso_acoes = []
volatilidade_carteira = []
sharpe_ratio = []
volatilidade_carteira_ajustada = []
sortino_ratio = []

In [299]:
# vamos usar uma simulação aleatória
numero_acoes = len(acoes)
numero_carteiras = 100000

np.random.seed(101)

In [300]:
retorno_diario = acoes_df.pct_change()
retorno_anual = retorno_diario.mean() * ANO_DIAS_UTEIS
cov_diaria = retorno_diario.cov()
cov_anual = cov_diaria * ANO_DIAS_UTEIS

retorno_negativo = retorno_diario[retorno_diario<0].dropna()
negativo_anual = retorno_negativo.mean() * ANO_DIAS_UTEIS
cov_diaria_neg = retorno_negativo.cov()
cov_neg_anual = cov_diaria_neg * ANO_DIAS_UTEIS


In [301]:
retorno_carteira = []
peso_acoes = []
volatilidade_carteira = []
sharpe_ratio = []
volatilidade_carteira_ajustada = []
sortino_ratio = []

for cada_carteira in range(numero_carteiras):
    taxa_livre_risco=SELIC
    pesos = np.random.random(numero_acoes)
    pesos /= np.sum(pesos)
    beta_portfolio = np.dot(pesos, list(betas.values()))
    retorno_portfolio = retorno_esperado(beta_portfolio,SELIC, IPCA)
    volatilidade_portfolio = np.sqrt(np.dot(pesos.T, np.dot(cov_anual, pesos)))
    sharpe = (retorno_portfolio - taxa_livre_risco) / volatilidade_portfolio
    volatilidade_ajustada = np.sqrt(np.dot(pesos.T, np.dot(cov_neg_anual, pesos)))
    sortino = (retorno_portfolio - taxa_livre_risco) / volatilidade_ajustada
    
    sharpe_ratio.append(sharpe)
    sortino_ratio.append(sortino)
    volatilidade_carteira.append(volatilidade_portfolio*100)
    volatilidade_carteira_ajustada.append(volatilidade_ajustada*100)
    peso_acoes.append(pesos*100)
    retorno_carteira.append(retorno_portfolio*100)
    

In [302]:
carteira = {'Retorno': retorno_carteira,
             'Volatilidade': volatilidade_carteira,
             'Sharpe Ratio': sharpe_ratio,
           'Vol Ajustada': volatilidade_carteira_ajustada,
           'Sortino Ratio': sortino_ratio}

for contar,acao in enumerate(acoes):
    carteira[acao+' Peso'] = [Peso[contar] for Peso in peso_acoes]

# vamos transformar nosso dicionário em um dataframe
df = pd.DataFrame(carteira)

# vamos nomear as colunas do novo dataframe
colunas = ['Retorno', 'Volatilidade', 'Sharpe Ratio', 'Vol Ajustada','Sortino Ratio' ] + [acao+' Peso' for acao in acoes]
df = df[colunas]

In [303]:
# vamos identificar as variáveis de interesse
menor_volatilidade = df['Volatilidade'].min()
maior_sharpe = df['Sharpe Ratio'].max()
maior_sortino = df['Sortino Ratio'].max()
menor_volatilidade_ajustada = df['Vol Ajustada'].min()
maior_retorno = df['Retorno'].max()

In [304]:
# vamos identificar os dois principais portfolios
carteira_sharpe = df.loc[df['Sharpe Ratio'] == maior_sharpe]
carteira_min_variancia = df.loc[df['Volatilidade'] == menor_volatilidade]
carteira_sortino = df.loc[df['Sortino Ratio'] == maior_sortino]
carteira_min_variancia_adj = df.loc[df['Vol Ajustada'] == menor_volatilidade_ajustada]
carteira_maiior_retorno= df.loc[df['Retorno'] == maior_retorno]

In [305]:
print("Essa é a carteira de Mínima Variância:", '\n', carteira_min_variancia.T)
print()
print("Essa é a carteira com maior Sharpe Ratio:", '\n', carteira_sharpe.T)
print()
print("Essa é a carteira com maior Sortino Ratio:", '\n', carteira_sortino.T)
print()
print("Essa é a carteira com menor Risco Ajustado:", '\n', carteira_min_variancia_adj.T)
print()
print("Maior Retorno:", '\n', carteira_maiior_retorno.T)

Essa é a carteira de Mínima Variância: 
                    43451
Retorno         3.479767
Volatilidade   39.966716
Sharpe Ratio   -0.088079
Vol Ajustada   33.637861
Sortino Ratio  -0.104651
MGLU3.SA Peso  24.084299
GGBR4.SA Peso  75.915701

Essa é a carteira com maior Sharpe Ratio: 
                    57876
Retorno         2.901796
Volatilidade   64.243815
Sharpe Ratio   -0.063791
Vol Ajustada   47.488005
Sortino Ratio  -0.086300
MGLU3.SA Peso  99.998867
GGBR4.SA Peso   0.001133

Essa é a carteira com maior Sortino Ratio: 
                    57876
Retorno         2.901796
Volatilidade   64.243815
Sharpe Ratio   -0.063791
Vol Ajustada   47.488005
Sortino Ratio  -0.086300
MGLU3.SA Peso  99.998867
GGBR4.SA Peso   0.001133

Essa é a carteira com menor Risco Ajustado: 
                    36024
Retorno         3.453217
Volatilidade   40.033368
Sharpe Ratio   -0.088596
Vol Ajustada   33.599026
Sortino Ratio  -0.105562
MGLU3.SA Peso  27.571520
GGBR4.SA Peso  72.428480

Maior Retorno: 
    

In [306]:
retorno_diario = acoes_df.pct_change()
retorno_anual = retorno_diario.mean() * ANO_DIAS_UTEIS
cov_diaria = retorno_diario.cov()
cov_anual = cov_diaria * ANO_DIAS_UTEIS

retorno_negativo = retorno_diario[retorno_diario<0].dropna()
negativo_anual = retorno_negativo.mean() * ANO_DIAS_UTEIS
cov_diaria_neg = retorno_negativo.cov()
cov_neg_anual = cov_diaria_neg * ANO_DIAS_UTEIS

retorno_carteira = []
peso_acoes = []
volatilidade_carteira = []
sharpe_ratio = []
volatilidade_carteira_ajustada = []
sortino_ratio = []

for cada_carteira in range(numero_carteiras):
    taxa_livre_risco=SELIC
    pesos = np.random.random(numero_acoes)
    pesos /= np.sum(pesos)
    #beta_portfolio = np.dot(pesos, list(betas.values()))
    retorno_portfolio = np.dot(pesos, retorno_anual)
    volatilidade_portfolio = np.sqrt(np.dot(pesos.T, np.dot(cov_anual, pesos)))
    sharpe = (retorno_portfolio - taxa_livre_risco) / volatilidade_portfolio
    volatilidade_ajustada = np.sqrt(np.dot(pesos.T, np.dot(cov_neg_anual, pesos)))
    sortino = (retorno_portfolio - taxa_livre_risco) / volatilidade_ajustada
    
    sharpe_ratio.append(sharpe)
    sortino_ratio.append(sortino)
    volatilidade_carteira.append(volatilidade_portfolio*100)
    volatilidade_carteira_ajustada.append(volatilidade_ajustada*100)
    peso_acoes.append(pesos*100)
    retorno_carteira.append(retorno_portfolio*100)
    

In [307]:
carteira = {'Retorno': retorno_carteira,
             'Volatilidade': volatilidade_carteira,
             'Sharpe Ratio': sharpe_ratio,
           'Vol Ajustada': volatilidade_carteira_ajustada,
           'Sortino Ratio': sortino_ratio}

for contar,acao in enumerate(acoes):
    carteira[acao+' Peso'] = [Peso[contar] for Peso in peso_acoes]

# vamos transformar nosso dicionário em um dataframe
df = pd.DataFrame(carteira)

# vamos nomear as colunas do novo dataframe
colunas = ['Retorno', 'Volatilidade', 'Sharpe Ratio', 'Vol Ajustada','Sortino Ratio' ] + [acao+' Peso' for acao in acoes]
df = df[colunas]

In [308]:
# vamos identificar as variáveis de interesse
menor_volatilidade = df['Volatilidade'].min()
maior_sharpe = df['Sharpe Ratio'].max()
maior_sortino = df['Sortino Ratio'].max()
menor_volatilidade_ajustada = df['Vol Ajustada'].min()
maior_retorno = df['Retorno'].max()

# vamos identificar os dois principais portfolios
carteira_sharpe = df.loc[df['Sharpe Ratio'] == maior_sharpe]
carteira_min_variancia = df.loc[df['Volatilidade'] == menor_volatilidade]
carteira_sortino = df.loc[df['Sortino Ratio'] == maior_sortino]
carteira_min_variancia_adj = df.loc[df['Vol Ajustada'] == menor_volatilidade_ajustada]
carteira_maiior_retorno= df.loc[df['Retorno'] == maior_retorno]

In [309]:
print("Essa é a carteira de Mínima Variância:", '\n', carteira_min_variancia.T)
print()
print("Essa é a carteira com maior Sharpe Ratio:", '\n', carteira_sharpe.T)
print()
print("Essa é a carteira com maior Sortino Ratio:", '\n', carteira_sortino.T)
print()
print("Essa é a carteira com menor Risco Ajustado:", '\n', carteira_min_variancia_adj.T)
print()
print("Maior Retorno:", '\n', carteira_maiior_retorno.T)

Essa é a carteira de Mínima Variância: 
                    92821
Retorno        18.862464
Volatilidade   39.966716
Sharpe Ratio    0.296809
Vol Ajustada   33.637799
Sortino Ratio   0.352653
MGLU3.SA Peso  24.087091
GGBR4.SA Peso  75.912909

Essa é a carteira com maior Sharpe Ratio: 
                    16937
Retorno        24.030187
Volatilidade   43.035260
Sharpe Ratio    0.395726
Vol Ajustada   35.945819
Sortino Ratio   0.473774
MGLU3.SA Peso   0.000220
GGBR4.SA Peso  99.999780

Essa é a carteira com maior Sortino Ratio: 
                    16937
Retorno        24.030187
Volatilidade   43.035260
Sharpe Ratio    0.395726
Vol Ajustada   35.945819
Sortino Ratio   0.473774
MGLU3.SA Peso   0.000220
GGBR4.SA Peso  99.999780

Essa é a carteira com menor Risco Ajustado: 
                    83226
Retorno        18.114903
Volatilidade   40.033367
Sharpe Ratio    0.277641
Vol Ajustada   33.599026
Sortino Ratio   0.330810
MGLU3.SA Peso  27.571490
GGBR4.SA Peso  72.428510

Maior Retorno: 
    